In [1]:
import pandas as pd
import pickle
import os
import time
import openai

# data 
# 2.3 million rows
drug_sub = pickle.load(open('../data/all_data_filtered_delauthor.pkl', 'rb'))
# 1,5 million
non_drug_sub = pickle.load(open('../data/non_drug_data_filtered_delauthor.pkl', 'rb'))

In [2]:
from openai import OpenAI
api_key = os.environ['OPENAI_API_KEY']
client = OpenAI(api_key = api_key)

---
## TASK1

In [3]:
from task1 import get_stigma_drug

In [4]:
seed = 1
drug_sample = drug_sub.sample(500, random_state = seed)
non_drug_sample = non_drug_sub.sample(500, random_state = seed)
dat = pd.concat([drug_sample, non_drug_sample])

In [6]:
dat['task1_label'] = dat['text'].apply(lambda x: get_stigma_drug(x, openai_client=client))

In [14]:
# save
pickle.dump(dat, open('../data/destigma_pipeline/task1_1.pkl', 'wb'))

---
## TASK 2

In [13]:
# label starts with 's-d'
dat_task2 = dat[dat['task1_label'].apply(lambda x: x.startswith('s-d'))]

In [15]:
pickle.dump(dat_task2, open('../data/destigma_pipeline/task2_1.pkl', 'wb'))

In [4]:
from task2 import get_utterance
task2 = pickle.load(open('../data/destigma_pipeline/task2/task2_1.pkl', 'rb'))

In [5]:
task2['task2_label'] = task2['text'].apply(lambda x: get_utterance(x, openai_client=client))

In [6]:
task2.to_csv('../data/evaluation_spreadsheets/task2_1.csv', index = False)

In [ ]:
# create loop for task 1 and task2
# chunks of 1000 rows 10 times

for i in range(10):
    print(i)
    seed = i
    drug_sample = drug_sub.sample(500, random_state = seed)
    non_drug_sample = non_drug_sub.sample(500, random_state = seed)
    dat = pd.concat([drug_sample, non_drug_sample])

    dat['task1_label'] = dat['text'].apply(lambda x: get_stigma_drug(x, openai_client=client))

    # save
    pickle.dump(dat, open('../data/destigma_pipeline/task1/task1_{}.pkl'.format(i), 'wb'))

    # label starts with 's-d'
    dat_task2 = dat[dat['task1_label'].apply(lambda x: x.startswith('s-d'))]
    dat_task2['task2_label'] = dat_task2['text'].apply(lambda x: get_stigma_drug(x, openai_client=client))

    # save
    pickle.dump(dat_task2, open('../data/destigma_pipeline/task2/task2_{}.pkl'.format(i), 'wb'))
    time.sleep(5) # sleep for 5 seconds